### 參考資料：
1. https://kernc.github.io/backtesting.py/
2. https://havocfuture.tw/blog/python-backtesting-py

### 零、安裝套件

In [ ]:
!pip install yfinance
!pip install backtesting

!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

### 壹、載入套件

In [ ]:
import yfinance as yf # 獲取股市API
import datetime as dt # 計時套件
import pandas as pd # 資料分析套件
from talib import abstract # 計算技術指標套件
from backtesting import Backtest, Strategy # 回測、策略套件
from backtesting.lib import crossover # 找交叉套件

### 貳、擷取股價

In [ ]:
# 輸入起始與結束時間
StartTime = dt.datetime(2010, 1, 1)
EndTime = dt.date.today()

# 輸入公司代號
CodeName = '2330.TW'

# 擷取資料
Data = yf.download(CodeName, start = StartTime, end = EndTime)

### 參、計算KD值

In [4]:
# 更改ta-lib所需格式
Data_Temp = Data.rename(columns={"High": "high", "Low": "low", "Close": "close"})

# 計算慢速隨機指標KD值
Data_KD = abstract.STOCH(Data_Temp, fastk_period = 9, slowk_period = 3, slowd_period = 3) # 慢速指標以3日移動平均值、快速指標以3日移動平均值

# 合併資料
Data = pd.merge(Data, Data_KD, on = "Date")

## 肆、回測

In [ ]:
class KD_Cross(Strategy):
    def init(self):
        super().init()

    def next(self):
        if crossover(20, self.data.slowk): # K < 20 買進
            self.buy()
        elif crossover(self.data.slowk, 80): # K > 80 賣出
            self.position.close()

bt = Backtest(Data, KD_Cross, cash = 10000, commission = 0.001425) # 交易手續費0.1425%
output = bt.run()
bt.plot()